#Food Delivery Dataset Preparation
This notebook integrates transactional order data, user master data, and restaurant master data into a single dataset that serves as the only source of truth for hackathon analysis.

In [1]:
#Import Required Libraries
import pandas as pd
import sqlite3


#Loading Dataset


In [2]:
# Load transactional order data
orders = pd.read_csv("orders.csv")

# Preview the data
orders.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [3]:
# Load user master data
users = pd.read_json("users.json")

# Preview the data
users.head()

,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [4]:
# Create SQLite database connection
conn = sqlite3.connect("restaurants.db")
cursor = conn.cursor()

# Execute SQL script to create and populate restaurant table
with open("restaurants.sql", "r") as file:
    sql_script = file.read()

cursor.executescript(sql_script)
conn.commit()

# Load restaurant data into pandas DataFrame
restaurants = pd.read_sql("SELECT * FROM restaurants", conn)

# Preview the data
restaurants.head()

,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


# Merge Datasets


In [5]:
# Merge orders with users using LEFT JOIN to retain all orders
orders_users = pd.merge(
    orders,
    users,
    on="user_id",
    how="left"
)

# Merge the result with restaurant data using LEFT JOIN
final_df = pd.merge(
    orders_users,
    restaurants,
    on="restaurant_id",
    how="left"
)


#Export Final Dataset

In [7]:
final_df.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [8]:
# drop the placeholder restaurant name
if "restaurant_name_y" in final_df.columns:
    final_df = final_df.drop(columns=["restaurant_name_y"])

# rename the real restaurant name properly
final_df = final_df.rename(columns={
    "restaurant_name_x": "restaurant_name",
    "name": "user_name"
})

final_df.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name,user_name,city,membership,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Italian,3.0


In [9]:
final_df = final_df[
    [
        "order_id",
        "order_date",
        "user_id",
        "user_name",
        "membership",
        "city",
        "restaurant_id",
        "restaurant_name",
        "cuisine",
        "rating",
        "total_amount"
    ]
]


In [11]:
final_df.to_csv("final_food_delivery_dataset.csv", index=False)

In [12]:
final_df.columns.tolist()


['order_id',
 'order_date',
 'user_id',
 'user_name',
 'membership',
 'city',
 'restaurant_id',
 'restaurant_name',
 'cuisine',
 'rating',
 'total_amount']

#MCQ Analysis
The following cells compute values used to answer the hackathon MCQs.

In [13]:
final_df[final_df["membership"] == "Gold"] \
    .groupby("city")["total_amount"] \
    .sum() \
    .sort_values(ascending=False)

,total_amount
city,
Chennai,1080909.79
Pune,1003012.32
Bangalore,994702.59
Hyderabad,896740.19


In [14]:
final_df.groupby("cuisine")["total_amount"].mean().sort_values(ascending=False)

,total_amount
cuisine,
Mexican,808.021344
Italian,799.448578
Indian,798.466011
Chinese,798.389020


In [15]:
user_totals = final_df.groupby("user_id")["total_amount"].sum()
(user_totals > 1000).sum()

np.int64(2544)

In [16]:
bins = [3.0, 3.5, 4.0, 4.5, 5.0]
labels = ["3.0–3.5", "3.6–4.0", "4.1–4.5", "4.6–5.0"]

final_df["rating_range"] = pd.cut(final_df["rating"], bins=bins, labels=labels)

final_df.groupby("rating_range")["total_amount"].sum().sort_values(ascending=False)

/tmp/ipython-input-884244713.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  final_df.groupby("rating_range")["total_amount"].sum().sort_values(ascending=False)


,total_amount
rating_range,
4.6–5.0,2197030.75
4.1–4.5,1960326.26
3.0–3.5,1881754.57
3.6–4.0,1717494.41


In [17]:
final_df[final_df["membership"] == "Gold"] \
    .groupby("city")["total_amount"] \
    .mean() \
    .sort_values(ascending=False)

,total_amount
city,
Chennai,808.459080
Hyderabad,806.421034
Bangalore,793.223756
Pune,781.162243


In [18]:
final_df.groupby("cuisine").agg(
    restaurant_count=("restaurant_id", "nunique"),
    total_revenue=("total_amount", "sum")
)



,restaurant_count,total_revenue
cuisine,,
Chinese,120,1930504.65
Indian,126,1971412.58
Italian,126,2024203.80
Mexican,128,2085503.09


In [19]:
gold_orders = (final_df["membership"] == "Gold").sum()
total_orders = len(final_df)

round((gold_orders / total_orders) * 100)

50

In [20]:
restaurant_stats = final_df.groupby("restaurant_name").agg(
    order_count=("order_id", "count"),
    avg_order_value=("total_amount", "mean")
)

restaurant_stats[restaurant_stats["order_count"] < 20] \
    .sort_values("avg_order_value", ascending=False)

,order_count,avg_order_value
restaurant_name,,
Hotel Dhaba Multicuisine,13,1040.222308
Sri Mess Punjabi,12,1029.180833
Ruchi Biryani Punjabi,16,1002.140625
Sri Delights Pure Veg,18,989.467222
Classic Kitchen Family Restaurant,19,973.167895
...,...,...
Annapurna Tiffins Punjabi,19,621.828947
Darbar Tiffins Non-Veg,18,596.815556
Darbar Restaurant Punjabi,14,589.972857


In [21]:
final_df.groupby(["membership", "cuisine"])["total_amount"] \
    .sum() \
    .sort_values(ascending=False)


membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

In [22]:
final_df["order_date"] = pd.to_datetime(final_df["order_date"], dayfirst=True)
final_df["quarter"] = final_df["order_date"].dt.to_period("Q")

final_df.groupby("quarter")["total_amount"].sum().sort_values(ascending=False)


,total_amount
quarter,
2023Q3,2037385.10
2023Q4,2018263.66
2023Q1,1993425.14
2023Q2,1945348.72
2024Q1,17201.50


In [23]:
gold_orders_count = final_df[final_df["membership"] == "Gold"].shape[0]
gold_orders_count


4987

In [24]:
hyderabad_revenue = final_df[final_df["city"] == "Hyderabad"]["total_amount"].sum()
round(hyderabad_revenue)



1889367

In [25]:
distinct_users = final_df["user_id"].nunique()
distinct_users


2883

In [26]:
gold_avg_order_value = final_df[final_df["membership"] == "Gold"]["total_amount"].mean()
round(gold_avg_order_value, 2)


np.float64(797.15)

In [27]:
high_rating_orders = final_df[final_df["rating"] >= 4.5].shape[0]
high_rating_orders


3374

In [30]:
# Step 1: find top revenue city among Gold members
top_gold_city = (
    final_df[final_df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .idxmax()
)

top_gold_city
# Step 2: count orders in that city (Gold members only)
orders_in_top_gold_city = final_df[
    (final_df["membership"] == "Gold") &
    (final_df["city"] == top_gold_city)
].shape[0]

orders_in_top_gold_city


1337

In [29]:
# Step 2: count orders in that city (Gold members only)



1337